In [1]:
import pandas
import numpy
import datetime
import feather
# import the lshell fit calc module
import os
import sys
module_path = os.path.abspath(os.path.join('/home/bharat/Documents/code/indsvels/py/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import saps_lshell_vel_map

In [2]:
# Some constants
eventDate = datetime.datetime(2015,2,3)
timeStart = datetime.datetime(2015,2,3,3,0)
timeEnd = datetime.datetime(2015,2,3,7,0)
sapsTime = datetime.datetime(2015,2,3,5,0)
sapsRadList = [ "cvw", "cve","fhw",\
               "fhe", "bks", "wal" ]
selBeam = 7
sapsMlatRange = [ 54., 63. ]#[ 56., 60. ]
sapsVelCutoff = 50.
sapsNpointsCutoff = 0 # per radar per beam
velScale = [ -500, 500 ]
velMagnScale = [ 0, 2000 ]

In [3]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT
0,20150203,100,0,17,-0.5291,-2.7541,56.3543,-113.0908,12.2337,59.4339,-173.7495,209,ade,2015-02-03 01:00:00,-11.7663
1,20150203,100,0,18,-0.3813,-1.0464,56.7910,-113.0603,12.2357,59.8341,-173.5086,209,ade,2015-02-03 01:00:00,-11.7643
2,20150203,100,0,19,-0.2573,0.6464,57.2264,-113.0327,12.2375,60.2326,-173.2659,209,ade,2015-02-03 01:00:00,-11.7625
3,20150203,100,0,20,-0.1531,1.0403,57.6608,-113.0076,12.2392,60.6293,-173.0208,209,ade,2015-02-03 01:00:00,-11.7608
4,20150203,100,0,21,-0.0657,1.6923,58.0945,-112.9849,12.2407,61.0246,-172.7729,209,ade,2015-02-03 01:00:00,-11.7593


In [4]:
# read from the feather file
velsDF = feather.read_dataframe('../data/saps-' +\
                eventDate.strftime("%Y%m%d") + '.feather')
velsDF["velMagn"] = numpy.abs(velsDF["vLos"])
# setup some fake vals
velsDF["spwdth"] = 50.
velsDF["pwr"] = 10.
# we dont need fits for all the times
velsDF = velsDF[ (velsDF["date"] >= timeStart) &\
               (velsDF["date"] <= timeEnd) ]
velsDF.head()

,dateStr,timeStr,beam,range,azimCalcMag,vLos,MLAT,MLON,MLT,GLAT,GLON,radId,radCode,date,normMLT,velMagn,spwdth,pwr
33885,20150203,300,0,17,-0.5291,-7.2099,56.3543,-113.0908,14.3791,59.4339,-173.7495,209,ade,2015-02-03 03:00:00,-9.6209,7.2099,50.0,10.0
33886,20150203,300,0,18,-0.3813,-8.0129,56.7910,-113.0603,14.3811,59.8341,-173.5086,209,ade,2015-02-03 03:00:00,-9.6189,8.0129,50.0,10.0
33887,20150203,300,0,19,-0.2573,-6.9616,57.2264,-113.0327,14.3829,60.2326,-173.2659,209,ade,2015-02-03 03:00:00,-9.6171,6.9616,50.0,10.0
33888,20150203,300,0,20,-0.1531,-5.5079,57.6608,-113.0076,14.3846,60.6293,-173.0208,209,ade,2015-02-03 03:00:00,-9.6154,5.5079,50.0,10.0
33889,20150203,300,0,21,-0.0657,-4.2073,58.0945,-112.9849,14.3861,61.0246,-172.7729,209,ade,2015-02-03 03:00:00,-9.6139,4.2073,50.0,10.0


In [5]:
allTimeList = velsDF["date"].unique()
fitResFrames = []
failedDTObjs = []
for timeSel in allTimeList:
    print timeSel
    inpVelDF = velsDF[ (velsDF["date"] == timeSel)\
                     ].reset_index(drop=True)
    lmObj = saps_lshell_vel_map.LshellMap( inpVelDF, timeSel )
    # get locations info for getting good fits
    azimCharDF = lmObj.azim_chars()
    # If no good fit is found discard and store the 
    # datetimes in a list for reference!
    if azimCharDF.shape[0] == 0:
        failedDTObjs.append(timeSel)
        print "no good azims found--->", timeSel
        continue
    # get actual good fits
    goodFitDF = lmObj.get_good_fits(azimCharDF)
    print "good fits", goodFitDF.shape
    # If no good fits are found! exit
    if goodFitDF.shape[0] == 0:
        print "no good fits found--->", timeSel
        continue
    # expand the fitting to cells with no fits
    fitResDF = lmObj.expand_fit_results(goodFitDF)
    print "expanding good fits", fitResDF.shape
    # Set up a few conditions to filter out some data
    # 1) If the number of fits is less than 5 skip
    if fitResDF.shape[0] < 5:
        continue
    # 2) number of good fits should be greater than bad one's
    if (fitResDF[ fitResDF["goodFit"] \
            ].shape[0]*1./fitResDF.shape[0]) <= 0.5:
        print "too many bad fits"
        continue
    # 3) Number of unique MLTs in the fits should be greater
    # than or equal to  (for example, 0, 1)
#     if len( fitResDF["normMlt"].unique().tolist() ) < 2.:
#         print "no large scale observations...."
#         continue
    # Need the datetime col as well
    fitResDF["date"] = timeSel
    fitResFrames.append( fitResDF )
# Merge all the good fit DFs
finFitDF = pandas.concat( fitResFrames )

2015-02-03T03:00:00.000000000


/home/bharat/Documents/code/indsvels/py/saps_lshell_vel_map.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfSel["rndAzim"] = dfSel[self.fitAzmType].round()
/home/bharat/Documents/code/indsvels/py/saps_lshell_vel_map.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x in dfSel["normMLT"] ]


good fits (0, 8)
no good fits found---> 2015-02-03T03:00:00.000000000
2015-02-03T03:02:00.000000000
good fits (2, 8)
expanding good fits (14, 10)
too many bad fits
2015-02-03T03:04:00.000000000
good fits (1, 8)
expanding good fits (14, 10)
too many bad fits
2015-02-03T03:06:00.000000000
good fits (5, 8)
expanding good fits (19, 10)
too many bad fits
2015-02-03T03:08:00.000000000
good fits (8, 8)
expanding good fits (18, 10)
too many bad fits
2015-02-03T03:10:00.000000000
good fits (12, 8)
expanding good fits (32, 10)
too many bad fits
2015-02-03T03:12:00.000000000
good fits (11, 8)
expanding good fits (33, 10)
too many bad fits
2015-02-03T03:14:00.000000000
good fits (10, 8)
expanding good fits (27, 10)
too many bad fits
2015-02-03T03:16:00.000000000
good fits (6, 8)
expanding good fits (21, 10)
too many bad fits
2015-02-03T03:18:00.000000000
good fits (3, 8)
expanding good fits (22, 10)
too many bad fits
2015-02-03T03:20:00.000000000
good fits (3, 8)
expanding good fits (21, 10)
too m

good fits (30, 8)
expanding good fits (44, 10)
2015-02-03T06:26:00.000000000
good fits (59, 8)
expanding good fits (71, 10)
2015-02-03T06:28:00.000000000
good fits (47, 8)
expanding good fits (70, 10)
2015-02-03T06:30:00.000000000
good fits (38, 8)
expanding good fits (54, 10)
2015-02-03T06:32:00.000000000
good fits (40, 8)
expanding good fits (59, 10)
2015-02-03T06:34:00.000000000
good fits (42, 8)
expanding good fits (67, 10)
2015-02-03T06:36:00.000000000
good fits (46, 8)
expanding good fits (77, 10)
2015-02-03T06:38:00.000000000
good fits (43, 8)
expanding good fits (72, 10)
2015-02-03T06:40:00.000000000
good fits (45, 8)
expanding good fits (78, 10)
2015-02-03T06:42:00.000000000
good fits (46, 8)
expanding good fits (77, 10)
2015-02-03T06:44:00.000000000
good fits (42, 8)
expanding good fits (84, 10)
too many bad fits
2015-02-03T06:46:00.000000000
good fits (48, 8)
expanding good fits (84, 10)
2015-02-03T06:48:00.000000000
good fits (59, 8)
expanding good fits (89, 10)
2015-02-03T

In [6]:
finFitDF['goodFit'] = [ 1 if x else 0 for x in finFitDF['goodFit'] ]
feather.write_dataframe( finFitDF, \
            "../data/lshellFits-" +\
            eventDate.strftime("%Y%m%d") + ".feather" )